In [25]:
#%matplotlib
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import join as opj
from numpy import median, rint, NA

from nipype.interfaces import freesurfer as fs
from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
project_home = '/Users/myelin/Documents/asl_practice'

# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = project_home + '/subs/freesurfer'
fs.FSCommand.set_default_subjects_dir(fs_dir)

#other study-specific variables
subjects_dir = project_home + '/subs'
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'
template = project_home + '/template/avg152T1.nii'

#pre-proc variables
numslices = 38
TR = 4.844
asl_slice_order = range(1,numslices+1)
middle_slice = np.around(median(asl_slice_order), decimals=-1)
middle_slice = middle_slice.astype(int)
#tpmfile = nib.load('~/spm12/tpm/TPM.nii')

#Population specific variables for ASL
T1_gm = 1.2 #T1 of gray matter tissue in seconds (for CBF quantification)
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.65 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header
postlabel_delay = 1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds

In [35]:
# preprocflow nodes

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subjid']),
                  name="infosource")
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'anat': subjects_dir + '/raw/{subjid}/anat.nii.gz',
              'func': subjects_dir + '/raw/{subjid}/cbf.nii.gz'}
selectfiles = Node(SelectFiles(templates), name="selectfiles")

#gunzip
gunzip = Node(Gunzip(), name="gunzip")

slice_timing = Node(fsl.SliceTimer(time_repetition=TR,
                               interleaved=False,output_type='NIFTI'),
                   name="slice_timing")

#smoothing- nt convinced we need this?
smooth = Node(spm.Smooth(), name="smooth")
smooth.inputs.fwhm = 4

#spatial normalization
norm = Node(spm.Normalize(template=template), name="norm")

# Datasink
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = subjects_dir
datasink.inputs.container = output_dir

def quantify_cbf(pw_nifti1,pd_nifti1,nex_asl,efficiency,T1_blood,sat_time,partition_coeff,scaling_factor,postlabel_delay,labeling_time):
    # set variables
    pw_data = pw_nifti1.get_data()
    pd_data = pd_nifti1.get_data()
    pd_nonzero_data = pd_data[pd_data==0]=[np.NA]
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    
    cbf_numerator = conversion*partition_coeff*pw_data*np.exp(postlabel_delay/T1_blood)
    cbf_denominator = nex_asl*efficiency*T1_blood*pd_data*(1-np.exp(-1*postlabel_delay/T1_blood))
    
    print(cbf_denominator)
    #cbf_data = cbf_numerator/cbf_denominator
    #cbf_vol = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    #nib.save(cbf_vol,project_home + '/cbf_vol.nii.gz')

In [36]:
pw_nifti1 = nib.load(project_home + '/subs/raw/ko/pw.nii.gz')
pd_nifti1 = nib.load(project_home + '/subs/raw/ko/pd.nii.gz')
quantify_cbf(pw_nifti1,pd_nifti1,nex_asl,efficiency,T1_blood,sat_time,partition_coeff,scaling_factor,postlabel_delay,labeling_time)


AttributeError: 'module' object has no attribute 'NA'

In [24]:
#establish asl preprocessing workflow that can be run on one subject at a time


preprocflow = Workflow(name="preprocflow")

preprocflow.base_dir = opj(subjects_dir)
preprocflow.connect([(infosource, selectfiles, [('subjid', 'subjid')]),
                     (selectfiles, gunzip, [('func', 'in_file')]),
                     (gunzip, slice_timing, [('out_file','in_file')]),
                     (slice_timing, datasink, [('slice_time_corrected_file', 'slice_timing')]),
                     (slice_timing, norm,[('slice_time_corrected_file','source')]),
                     (norm, datasink, [('normalized_source', 'norm')]),
                    ])
preprocflow.write_graph(graph2use='flat')
preprocflow.run('MultiProc', plugin_args={'n_procs': 2})

170225-16:04:06,320 workflow INFO:
	 Creating detailed dot file: /Users/myelin/Documents/asl_practice/subs/preprocflow/graph_detailed.dot
170225-16:04:07,331 workflow INFO:
	 Creating dot file: /Users/myelin/Documents/asl_practice/subs/preprocflow/graph.dot
170225-16:04:08,431 workflow INFO:
	 ['check', 'execution', 'logging']
170225-16:04:08,514 workflow INFO:
	 Running in parallel.
170225-16:04:08,526 workflow INFO:
	 Executing: selectfiles.a0 ID: 0
170225-16:04:08,546 workflow INFO:
	 Executing node selectfiles.a0 in dir: /Users/myelin/Documents/asl_practice/subs/preprocflow/_subjid_ko/selectfiles
170225-16:04:08,629 workflow INFO:
	 Runtime memory and threads stats unavailable
170225-16:04:08,634 workflow INFO:
	 [Job finished] jobname: selectfiles.a0 jobid: 0
170225-16:04:08,637 workflow INFO:
	 Executing: gunzip.a0 ID: 1
170225-16:04:08,641 workflow INFO:
	 [Job finished] jobname: gunzip.a0 jobid: 1
170225-16:04:08,644 workflow INFO:
	 Executing: slice_timing.a0 ID: 2
170225-16:0

In [ ]:
#establish processing workflow for analysis of multiple subjects
procflow = Workflow(name="procflow")


In [ ]:
#proc nodes


In [ ]:
modelflow = Workflow(name="modelflow")